**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json
import os

from keras.models import Sequential,model_from_json, Model
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization, Flatten, Input, Dense


C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
print(skvideo.__file__)

C:\ProgramData\Anaconda3\lib\site-packages\skvideo\__init__.py


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [3]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [4]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
        self.dec_value = 0
        
    def set_epsilon(self,e):
        # For question 10. I increase a factor that involves decreasing the value of epsilon.
        self.epsilon = self.epsilon/np.exp(self.dec_value)
        self.dec_value -= 0.5 # I decrease the value for the next time. It starts in 0

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

As we know, epsilon here represents the probability of doing "exploration" of "explotation". In Reinforcement Learning we constantly work with these two because there is always a trade off between "exploration" of "explotation".
 
One of the challenges that arise in reinforcement learning is the trade-off between exploration and exploitation. To obtain a lot of reward, a reinforcement learning agent must prefer actions that it has tried in the past and found to be effective in producing reward. But to discover such actions, it has to try actions that it has not selected before (like exploration)

The agent has to exploit what it has already experienced in order to obtain reward, but it also has to explore in order to make better action selections in the future. The dilemma is that neither exploration nor exploitation can be pursued exclusively without failing at the task. The agent must try a variety of actions and progressively favor those that appear to be best.

On a stochastic task, each action must be tried many times to gain a reliable estimate of its expected reward. We are trying to balance the exploration–exploitation with the epsilon parameter to ensure that with certain probability the agent explores new options, or exploits one that he already knows.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [5]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [6]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train= 11 # set small when debugging
epochs_test= 11 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

"Board" in this case represents in our reinforcement learning model as the "Environment" because it considers the delimitation of the island (NxN island where N = grid_size + 4), the building of the walls of the islands (the two first rows and columns that surround the island) and the information of the poison mushrooms and cheese.

"Position Board" is consider as the agent because it has the status of the board but saving the position where the agent (rat) is. It will save its position and then keep updating it for each moves it does.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [7]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        # If it's a random agent, we just need to get a random value (action) and perform it.
        action = np.random.randint(0, self.n_action, size=1)[0]
        return action

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [8]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        #################
        ##### FILL IN HERE
        # Based on the agent defined at the beginning of the homework, we build this structure.
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the game will end
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.learned_act(state)

            # Apply an action to the environment, get the next state, the reward
            # and check if the games end (we pass the time)
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        
        
        ################
        
        # Save as a mp4
        if e == 10:
            env.draw(prefix+str(e))

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [9]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random10.mp4'))

Win/lose count 9.5/15.0. Average score (-5.5)
Win/lose count 11.5/7.0. Average score (-0.5)
Win/lose count 6.5/8.0. Average score (-0.8333333333333334)
Win/lose count 11.5/14.0. Average score (-1.25)
Win/lose count 8.5/14.0. Average score (-2.1)
Win/lose count 10.5/16.0. Average score (-2.6666666666666665)
Win/lose count 12.5/20.0. Average score (-3.357142857142857)
Win/lose count 8.0/10.0. Average score (-3.1875)
Win/lose count 8.0/19.0. Average score (-4.055555555555555)
Win/lose count 11.0/13.0. Average score (-3.85)
Win/lose count 9.0/12.0. Average score (-3.772727272727273)
Final score: -3.772727272727273


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




> ### Question 5.1:
> 
>We know that the Action Value function is given by:
>$Q^{\pi}(s, a) = E_{\pi}[R_{t} | S_{t}=s, A_{t}=a]$
>
>And the Reward function is:
>$R_{t} = \sum_{k=t+1}^{T} \gamma^{k-t-1} r_{k}$, $r_{k}$ is the reward in time k
>
>We work on them and get the following:
>
>$$Q^{\pi}(s, a)={E}^{\pi}[\sum_{k=0}^{\infty} \gamma^{k} r_{t+k+1} | S_{t}=s, A_{t}=a]$$ 
>$$ = {E}^{\pi}[r(s,a) + \sum_{k=1}^{\infty} \gamma^{k} r_{t+k+1} | S_{t}=s, A_{t}=a]$$
>$$ = {E}^{\pi}[r(s,a) +\gamma Q^{\pi}(s',a') ] $$
>$$ = E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$$


>### Question 5.2:
> By monotonicity property in reinforcement learning, we know that we must get better rewards when we do more actions, until we arrive to the optimal action value function. For this, we know that $\pi'$ has to be bigger than $\pi$ for the next state. Also, our value function $v_{\pi'}(s)$ will be bigger than our previous $v_{\pi}(s)$. If we go to the infinity in time, we arrive to the optimal action value function. From our previous equation, we get that:
>
> $$Q^{*}(s,a) = \max _{a}E^{\pi^{*}}[R_{t}| S_{t}=s, A_{t}=a]$$
> $$ = \max _{a}E^{\pi^{*}}[r(s,a) + R_{t+1}| S_{t}=s, A_{t}=a]$$
> $$ = E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]$$

>### Question 5.3:
>The loss function is defined as the distance between the predicted action value function $Q$ that we have in our current state and the optimal action value function $Q^{*}$. Both are given in previous questions. Based on this we got:
>
>$$\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert Q^{*}(s,a) - Q^{\pi}(s, a)  \Vert^{2}$$
>$$=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}$$

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [10]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        # We include the value "m" that has the array of form: [s_, n_s_, a_, r_, game_over_].
        # State, New_state, Action, Reward, Game_Over
        if len(self.memory) < self.max_memory:
            self.memory.append(m)
        else: # If we reach the maximum amount of memory, we pop the 1st value of the array and put the append the new one. FIFO method.
            self.memory.pop(0)
            self.memory.append(m)

    def random_access(self):
        # We want to get a random value from the memory. We create a variable that considers the length of the memory array.
        max_memory_box = len(self.memory) # No need of "-1" because randint is exclusive interval on the right.
        # This value considers the array of [s_, n_s_, a_, r_, game_over_]. It gets it from the memory array with its index.
        mem = self.memory[np.random.randint(0, max_memory_box)]#, size=1)]
        return mem

***
The pipeline we will use for training is given below:

In [11]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e==10:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [12]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size
        
        # Actions that can be performed
        self.actions = [0,1,2,3]

    def learned_act(self, s):
        # We reshape based on the 3D matrix of states: 5x5x2
        s = s.reshape(1,s.shape[0], s.shape[1], s.shape[2])
        # We predict based on the state we have.
        np_predict = self.model.predict(s)
        # We maximize our state based on the result of predictions we got and get our final state.
        final_state_value = np.argmax(np_predict)
        return final_state_value

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        input_states = np.zeros((self.batch_size, 5,5,self.n_state))
        
        # I define the shape of our target Q considering the batch.
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            ######## FILL IN
            
            # I get the info of historical state (in column 0) that we have in memory to learn from the pool.
            hist_state = self.memory.random_access()[0]
            # I get the info of historical reward (in column 3) that we have in memory to learn from the pool.
            hist_reward = self.memory.random_access()[3]
            # I reshape the historical state  to our 5X5X2
            hist_state = hist_state.reshape(1,hist_state.shape[0], hist_state.shape[1], hist_state.shape[2])
            
            # Based on the action value function, we define Q as the actual reward (hist_reward) + the future Qs that we will have multiplied by discount (gamma)
            for j in range(4):
                target_q[i,j] = hist_reward + self.discount*self.model.predict(hist_state).reshape(4)[j]
            
            # Actually the value "game_over" is updated when we apply an action in the environment (every round) and we get out of time.
            if game_over_:
                game_over = True
                ######## FILL IN
            else:
                game_over = False
                ######## FILL IN
        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)        
        l = self.model.train_on_batch(input_states, target_q)

        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # I tried with a Dense Sequential Model, but it tells me there is a problem with the dimensions, despite they are good.
        
        #model = Sequential()
        #model.add(Conv2D(filters=1,kernel_size=(3,3), input_shape=(5,5,2)))
        #model.add(Flatten())
        #model.add(Dense(units=64, activation='relu'))
        #model.add(Dense(units=128, activation='relu'))
        #model.add(Dense(units=len(self.actions))
        
        # I tried also with Convolutional network, but because it's sequential again, i got problems with dimensions.
        
        # Initializes a sequential model
        #model = Sequential()
        # First layer
        #model.add(Conv2D(2, kernel_size = (5,5), activation='relu', batch_input_shape=(32,5,5,2), data_format = 'channels_last'))
        #model.add(Dense(1024, activation='relu', batch_input_shape=(32,5,5,2), data_format = 'channels_last'))
        # We include the Pool of 2x2 for the model
        #model.add(MaxPooling2D(2))
        # We convert it to on single big array when we go from Convolution to Dense.
        #model.add(Flatten())
        # Output layer
        #model.add(Dense(len(self.actions), activation='linear', batch_size = 32))
        
        # This model worked. I build manually the layers (not sequential) and associate the inputs to the outputs manually.
        # The model considers the b atch defined in 32.
        a = Input(batch_shape=(32,5,5,self.n_state))
        b = Flatten()(a)
        c = Dense(64, activation='relu')(b)
        d = Dense(4, activation='linear')(c) # We finish in four because of the quantity of actions.
        model = Model(inputs=a, outputs=d)
        
        ####### FILL IN
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        model.summary()

In [13]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (32, 5, 5, 2)             0         
_________________________________________________________________
flatten_1 (Flatten)          (32, 50)                  0         
_________________________________________________________________
dense_1 (Dense)              (32, 64)                  3264      
_________________________________________________________________
dense_2 (Dense)              (32, 4)                   260       
Total params: 3,524
Trainable params: 3,524
Non-trainable params: 0
_________________________________________________________________
Epoch 000/011 | Loss 0.1983 | Win/lose count 2.5/4.0 (-1.5)
Epoch 001/011 | Loss 0.0756 | Win/lose count 2.0/4.0 (-2.0)
Epoch 002/011 | Loss 0.0293 | Win/lose count 2.0/2.0 (0.0)
Epoch 003/011 | Loss 0.0457 | Win/lose count 2.5/5.0 (-2.5)
Epoch 004/011 | Loss 0.0324 | Win

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [14]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        # Based on the model applied before, i build it again but now with convolutional layers as requested.
        # This model considers that batch also as 32.
        a = Input(batch_shape=(32,5,5,self.n_state))
        conv = Conv2D(64, kernel_size = (3,3), activation='relu', data_format = 'channels_last')(a)
        conv2 = Conv2D(128, kernel_size = (3,3), activation='relu', data_format = 'channels_last')(conv)
        b = Flatten()(conv)
        c = Dense(64, activation='relu')(b)
        d = Dense(4, activation='relu')(c)
        model = Model(inputs=a, outputs=d)
        
        ###### FILL IN
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model
        model.summary()

In [15]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (32, 5, 5, 2)             0         
_________________________________________________________________
conv2d_1 (Conv2D)            (32, 3, 3, 64)            1216      
_________________________________________________________________
flatten_2 (Flatten)          (32, 576)                 0         
_________________________________________________________________
dense_3 (Dense)              (32, 64)                  36928     
_________________________________________________________________
dense_4 (Dense)              (32, 4)                   260       
Total params: 38,404
Trainable params: 38,404
Non-trainable params: 0
_________________________________________________________________
Epoch 000/011 | Loss 0.0060 | Win/lose count 4.5/5.0 (-0.5)
Epoch 001/011 | Loss 0.0342 | Win/lose count 2.5/5.0 (-2.5)
Epoch 002/011 | Lo

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [16]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (32, 5, 5, 2)             0         
_________________________________________________________________
conv2d_3 (Conv2D)            (32, 3, 3, 64)            1216      
_________________________________________________________________
flatten_3 (Flatten)          (32, 576)                 0         
_________________________________________________________________
dense_5 (Dense)              (32, 64)                  36928     
_________________________________________________________________
dense_6 (Dense)              (32, 4)                   260       
Total params: 38,404
Trainable params: 38,404
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_

In [17]:
# We include one more cell to see the difference of the temperature. In this case we increase the temperature from 0.3 to 0.9.
env = Environment(grid_size=size, max_time=T,temperature=0.9)
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         (32, 5, 5, 2)             0         
_________________________________________________________________
conv2d_5 (Conv2D)            (32, 3, 3, 64)            1216      
_________________________________________________________________
flatten_5 (Flatten)          (32, 576)                 0         
_________________________________________________________________
dense_9 (Dense)              (32, 64)                  36928     
_________________________________________________________________
dense_10 (Dense)             (32, 4)                   260       
Total params: 38,404
Trainable params: 38,404
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_

In [18]:
HTML(display_videos('cnn_test10.mp4'))

In [19]:
HTML(display_videos('fc_test10.mp4'))

Normally in these tests, the CNN has been getting better results, but they are almost the same as the FC (sometimes one gets bigger than the other, but not always). Also, I can see that the CNN model delays a bit more than the FCC in processing. We see that the mouse has a rare behaviour when he's in a place where there are no more rewards or mushrooms. He comes and goes so he mantains safe and it's not exploring too much (or goes around before doing it).

If we check the temperature variable, when we increase it from 0.3 to 0.9 (the other cell added), we can see that the results improved in both models and the average results in negative decreased. I deduct that this is because we when temperature increases, there is more options or getting rewards (also mushrooms) but it helps us to get more positive points to improve our overall performance.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [20]:
# Based on what the book "An Introduction to Reinforcement Learning", the e-greedy methods take more exploration
# to find the optimal action with noisy rewards (in our case the mushrooms), but they are better than the greedy
# methods. Also it mentions: "The e = 0.1 method explored more, and usually found the optimal action earlier, but
# it never selected that action more than 91% of the time. The e = 0.01 method improved more slowly, but eventually
# would perform better than the e = 0.1 method on both performance measures".

# Considering this, I will apply a decreasing epsilon based on how time passes. This is applied in the "set_epsilon" function.

def train_explore(agent,env,epoch,prefix=''):
# Using the previous model we got from previous questions, I build the train function.
# Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        # I call the agent to work with the epsilon that will vary every time I set the epsilon.
        agent.set_epsilon(agent.epsilon)
        
        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e==10:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

## use those samples of code:
#In train explore:
#state, reward, game_over = env.act(action, train=True)

class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        
        # Adding the board for the visited cells
        self.malus_position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train = True):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:,-2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        
        # I add here the additional state matrix considering if it's visited or not.
        
        ########### Inclusion for visit cell ##########
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1 #I registered the value to show this cell was visited
        reward = reward + self.board[self.x, self.y]
        ##############################################
        
        #reward = self.board[self.x, self.y]        
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        
        # I include the malus_position matrix in the state matrix to see which was visited.
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
            self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[:, -2:] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        # I include the malus_position matrix in the state matrix to see which was visited.
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
#reward = 0
#if train:
#    reward = -self.malus_position[self.x, self.y]
#self.malus_position[self.x, self.y] = 0.1

#reward = reward + self.board[self.x, self.y]
## 3 "feature" states instead of 2
#state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
#                                self.board.reshape(self.grid_size, self.grid_size,1),
#                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

In [21]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32,n_state=3)
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (32, 5, 5, 3)             0         
_________________________________________________________________
conv2d_7 (Conv2D)            (32, 3, 3, 64)            1792      
_________________________________________________________________
flatten_7 (Flatten)          (32, 576)                 0         
_________________________________________________________________
dense_13 (Dense)             (32, 64)                  36928     
_________________________________________________________________
dense_14 (Dense)             (32, 4)                   260       
Total params: 38,980
Trainable params: 38,980
Non-trainable params: 0
_________________________________________________________________
Epoch 000/011 | Loss 0.0316 | Win/lose count 3.5/22.800000000000065 (-19.300000000000065)
Epoch 001/011 | Loss 0.0489 | Win/lose count 4.6

In [22]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

Win/lose count 0.4/21.00000000000003. Average score (-20.60000000000003)
Win/lose count 0.4/22.000000000000043. Average score (-21.100000000000037)
Win/lose count 0.4/20.000000000000014. Average score (-20.60000000000003)
Win/lose count 0.8/19.900000000000013. Average score (-20.225000000000026)
Win/lose count 1.2000000000000002/21.80000000000004. Average score (-20.30000000000003)
Win/lose count 0.4/20.000000000000014. Average score (-20.18333333333336)
Win/lose count 0.4/21.000000000000032. Average score (-20.24285714285717)
Win/lose count 0/21.100000000000033. Average score (-20.350000000000026)
Win/lose count 0.4/21.000000000000032. Average score (-20.377777777777805)
Win/lose count 0/21.100000000000033. Average score (-20.450000000000024)
Win/lose count 0.4/20.000000000000014. Average score (-20.372727272727296)
Final score: -20.372727272727296


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***